<a href="https://colab.research.google.com/github/Om-Shankar-Thakur/fake_news_prediction/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article ; could be incomplete

label: a label that marks whether the news article is real or fake

1: fake news

0: real news

In [1]:
import pandas as pd
import numpy as np
import re             #used for searching words in a text or paragraph
from nltk.corpus import stopwords    #those words which doesn't add value to a paragraph like articles, a, etc
from nltk.stem.porter import PorterStemmer  #it gives the root word for a particular word
from sklearn.feature_extraction.text import TfidfVectorizer   #used to convert this text into feature vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data preprocessing


In [4]:
news_dataset = pd.read_csv('/content/train.csv')

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
#replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [24]:
news_dataset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [25]:
#merging the author name and news title
news_dataset['content'] =  news_dataset['author'] + ' '+news_dataset['title']

In [26]:
news_dataset.head

<bound method NDFrame.head of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2        

In [27]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [28]:
#seperating the data and label
X = news_dataset.drop(columns='label', axis =1)
Y = news_dataset['label']

In [29]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

it is the processs of reducing a word to its root word

example:
actor, actress, acting --> act

In [30]:
port_stem = PorterStemmer()

In [31]:
def stemming(content):
  stemmed_content = re.sub('[˄a-zA-Z]', ' ', content)   #removes everything (which is not alphabet) with content
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split() #converts to a list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]  #removes all the stopwords
  stemmed_content = ' '.join(stemmed_content)   #join all words
  return stemmed_content

In [32]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [33]:
print(news_dataset['content'])

0              : ’ ’
1            . : , -
2                  .
3                 15
4                   
            ...     
20795       . .: ’ ’
20796    . . . : , -
20797        . ’ ’ -
20798              ,
20799            -35
Name: content, Length: 20800, dtype: object


In [34]:
#seperating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [35]:
print(X)

[': ’ ’' '. : , -' '.' ... '. ’ ’ -' ',' '-35']


In [36]:
print(Y)

[1 0 1 ... 0 1 1]


In [37]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [38]:
print(X)

  (3, 62)	1.0
  (27, 62)	1.0
  (31, 145)	0.4249057758414498
  (31, 20)	0.49454392483696136
  (31, 7)	0.7582093299765108
  (36, 146)	0.7333171625424529
  (36, 131)	0.6798867105045412
  (37, 220)	0.8630627144132533
  (37, 145)	0.5050967738855863
  (40, 131)	1.0
  (47, 1204)	0.3691999620908306
  (47, 1124)	0.38440673900713157
  (47, 1058)	0.3691999620908306
  (47, 1040)	0.3691999620908306
  (47, 1014)	0.38440673900713157
  (47, 939)	0.38440673900713157
  (47, 869)	0.38440673900713157
  (53, 1386)	1.0
  (56, 251)	1.0
  (61, 47)	1.0
  (73, 348)	1.0
  (75, 263)	1.0
  (76, 263)	0.7876361228177006
  (76, 1)	0.6161406803910127
  (78, 243)	1.0
  :	:
  (20680, 1)	0.6161406803910127
  (20690, 1097)	0.3159830601044173
  (20690, 1089)	0.3159830601044173
  (20690, 1079)	0.3159830601044173
  (20690, 893)	0.3159830601044173
  (20690, 800)	0.2244087729413534
  (20690, 784)	0.2244087729413534
  (20690, 781)	0.3159830601044173
  (20690, 734)	0.3034830609713406
  (20690, 706)	0.2663660348465984
  (20690, 6

splitting the dataset

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

training the model:  LOGISTIC REGRESSION

In [41]:
model = LogisticRegression()

In [42]:
model.fit(X_train, Y_train)

LogisticRegression()

evaluation


accuracy score

In [44]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [45]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.5409254807692307
